In [1]:
#imports
import sys
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from tensorflow.keras.metrics import Precision, Recall
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import OrdinalEncoder

# Zadanie rekrutacyjne Onwel

## Treść Zadania:

1. Skopiuj repozytorium na swoją maszynę: ML_Academy. Wczytaj projekt na
swoje konto git.
2. Wczytaj zbiór danych, nadaj labelki, ustal taktykę dzielnia zbioru do
treningu.
3. Zbuduj pipeline przygotowujący dane do treningu / predykcji
4. Wytrenuj model / modele
5. Wybierz i przedstaw metryki ewaluacji modelu
6. Przedstaw i opisz, jak model będzie sobie radził na danych
&quot;produkcyjnych&quot; (takich, które nie były dostępne na czas treningu /
ewaluacji)

## 1. Adres repozytorium - https://github.com/MarcinosP/onwel

## 2. Wczytaj zbiór danych, nadaj labelki, ustal taktykę dzielnia zbioru do treningu.

### połączenie z dyskiem google

In [2]:
drive.mount('/content/drive')
path_nb = r'/content/drive/My Drive/Colab Notebooks/onwel'
sys.path.append(path_nb)

Mounted at /content/drive


### wczytanie danych

In [3]:
benign_data = pd.read_csv(path_nb+'/CSV_benign.csv', low_memory=False)
malware_data = pd.read_csv(path_nb+'/CSV_malware.csv', low_memory=False)

benign_data['label'] = 0
malware_data['label'] = 1

combined_data = pd.concat([benign_data, malware_data], ignore_index=True)

### sprawdzenie ile zbiór danych zawiera wartości null, zastąpenie wartości null dla cech tesktowych wartością "unkown" a dla cech numerycznych najczęściej występującą wartością

In [4]:
missing_values = combined_data.isnull().sum()

missing_values

Country               109887
ASN                   109830
TTL                        0
IP                    108596
Domain                     0
State                 278997
Registrant_Name       485424
Country.1             242151
Creation_Date_Time    133088
hex_32                   123
hex_8                      0
Domain_Name            97728
Alexa_Rank             54418
subdomain                 16
Organization          295105
len                      358
longest_word             133
oc_32                      3
shortened               1345
1gram                     22
obfuscate_at_sign          0
entropy                    4
Domain_Age             54414
tld                        3
dec_8                     10
dec_32                    94
Emails                172234
numeric_percentage       789
puny_coded                 0
typos                      0
oc_8                       0
3gram                      0
char_distribution          0
2gram                    187
Registrar     

In [5]:
text_columns = combined_data.select_dtypes(include=['object']).columns
combined_data[text_columns] = combined_data[text_columns].fillna("unknown")

numeric_columns = combined_data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    mode_val = combined_data[col].mode()[0]
    combined_data[col].fillna(mode_val, inplace=True)

In [6]:
missing_values = combined_data.isnull().sum()

missing_values

Country               0
ASN                   0
TTL                   0
IP                    0
Domain                0
State                 0
Registrant_Name       0
Country.1             0
Creation_Date_Time    0
hex_32                0
hex_8                 0
Domain_Name           0
Alexa_Rank            0
subdomain             0
Organization          0
len                   0
longest_word          0
oc_32                 0
shortened             0
1gram                 0
obfuscate_at_sign     0
entropy               0
Domain_Age            0
tld                   0
dec_8                 0
dec_32                0
Emails                0
numeric_percentage    0
puny_coded            0
typos                 0
oc_8                  0
3gram                 0
char_distribution     0
2gram                 0
Registrar             0
sld                   0
Name_Server_Count     0
Page_Rank             0
label                 0
dtype: int64

### Użycie label encoding na wartościach tekstowych.

In [7]:
unique_values_counts = combined_data.nunique()
unique_values_counts

Country                  285
ASN                    10356
TTL                    12864
IP                    177855
Domain                313538
State                   5813
Registrant_Name         4203
Country.1               1620
Creation_Date_Time    145659
hex_32                   124
hex_8                     20
Domain_Name           195952
Alexa_Rank            114288
subdomain                186
Organization           39736
len                      515
longest_word           34203
oc_32                   7205
shortened                 98
1gram                 211753
obfuscate_at_sign      13501
entropy                 4296
Domain_Age            333589
tld                    31786
dec_8                     77
dec_32                   164
Emails                 44745
numeric_percentage      9963
puny_coded               185
typos                  30268
oc_8                     926
3gram                 210746
char_distribution     284881
2gram                 229924
Registrar     

In [8]:
column_data_types = combined_data.dtypes
column_data_types

Country                object
ASN                   float64
TTL                    object
IP                     object
Domain                 object
State                  object
Registrant_Name        object
Country.1              object
Creation_Date_Time     object
hex_32                 object
hex_8                  object
Domain_Name            object
Alexa_Rank             object
subdomain              object
Organization           object
len                    object
longest_word           object
oc_32                  object
shortened              object
1gram                  object
obfuscate_at_sign      object
entropy                object
Domain_Age             object
tld                    object
dec_8                  object
dec_32                 object
Emails                 object
numeric_percentage     object
puny_coded             object
typos                  object
oc_8                   object
3gram                  object
char_distribution      object
2gram     

In [9]:
categorical_columns_all = combined_data.select_dtypes(include=['object']).columns.tolist()

label_encoders_combined = {}

for col in categorical_columns_all:
    le = LabelEncoder()
    combined_data[col] = le.fit_transform(combined_data[col].astype(str))
    label_encoders_combined[col] = le

In [10]:
combined_data_cleaned = combined_data.drop(columns=categorical_columns_all)
final_combined_data = pd.concat([combined_data_cleaned, combined_data[categorical_columns_all]], axis=1)

### Normalizacja danych za pomocą min max scaler do wartości od 0 do 1

In [11]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(final_combined_data.drop(columns='label'))
final_combined_scaled_data = pd.DataFrame(scaled_data, columns=final_combined_data.drop(columns='label').columns)

### Podział danych na testowe i treningowe 80:20 z użyciem stratify które pomaga zachować rozkład klas w zbiorach testowych i treningowych

In [12]:
X = final_combined_scaled_data
y = final_combined_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 4. trenowanie modeli

In [13]:
n_features = final_combined_scaled_data.shape[1]

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(n_features,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
12479/12479 [==============================] - 34s 3ms/step - loss: 0.0539 - accuracy: 0.9898
Epoch 2/10
12479/12479 [==============================] - 33s 3ms/step - loss: 0.0477 - accuracy: 0.9901
Epoch 3/10
12479/12479 [==============================] - 32s 3ms/step - loss: 0.0461 - accuracy: 0.9902
Epoch 4/10
12479/12479 [==============================] - 32s 3ms/step - loss: 0.0454 - accuracy: 0.9902
Epoch 5/10
12479/12479 [==============================] - 31s 3ms/step - loss: 0.0446 - accuracy: 0.9902
Epoch 6/10
12479/12479 [==============================] - 32s 3ms/step - loss: 0.0444 - accuracy: 0.9902
Epoch 7/10
12479/12479 [==============================] - 33s 3ms/step - loss: 0.0441 - accuracy: 0.9902
Epoch 8/10
12479/12479 [==============================] - 32s 3ms/step - loss: 0.0437 - accuracy: 0.9903
Epoch 9/10
12479/12479 [==============================] - 32s 3ms/step - loss: 0.0436 - accuracy: 0.9903
Epoch 10/10
12479/12479 [==============================

In [16]:
predictions = model.predict(X_test)

3120/3120 [==============================] - 6s 2ms/step


In [17]:
binary_predictions = (predictions > 0.5).astype(int)

In [18]:
accuracy = accuracy_score(y_test, binary_predictions)
print(f"accuracy {accuracy} ")

precision = precision_score(y_test, binary_predictions)
print(f"precision {precision} ")

recall = recall_score(y_test, binary_predictions)
print(f"recall {recall} ")

f1 = f1_score(y_test, binary_predictions)
print(f"f1 {f1} ")

conf_matrix = confusion_matrix(y_test, binary_predictions)
print(f"conf_matrix {conf_matrix} ")

accuracy 0.9900928606489227 
precision 1.0 
recall 0.011 
f1 0.021760633036597428 
conf_matrix [[98827     0]
 [  989    11]] 


### z recall, f1 score i macierzy pomyłek  możemy odczytać że model sklasyfikował zdecydowaną większość jako "benign" wynika to z nierównomiernej dystrybucji klas.

###Dlatego poniżej powtórzę powyższe kroki wyrównując dystrybucje klas poprzez powielenie danych które są wirusami tyle aż razy dystrybucja klas będzie taka sama, jedynie dla danych treningowych

In [19]:
oversampled_data = pd.concat([benign_data, malware_data], ignore_index=True)

In [20]:
text_columns = oversampled_data.select_dtypes(include=['object']).columns
oversampled_data[text_columns] = oversampled_data[text_columns].fillna("unknown")

numeric_columns = oversampled_data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    mode_val = oversampled_data[col].mode()[0]
    oversampled_data[col].fillna(mode_val, inplace=True)

In [21]:
label_encoders_combined = {}

for col in categorical_columns_all:
    le = LabelEncoder()
    oversampled_data[col] = le.fit_transform(oversampled_data[col].astype(str))
    label_encoders_combined[col] = le

In [22]:
os_combined_data_cleaned = oversampled_data.drop(columns=categorical_columns_all)
os_final_combined_data = pd.concat([os_combined_data_cleaned, oversampled_data[categorical_columns_all]], axis=1)

In [23]:
os_scaler = MinMaxScaler()
os_scaled_data = os_scaler.fit_transform(os_final_combined_data.drop(columns='label'))
os_final_combined_scaled_data = pd.DataFrame(os_scaled_data, columns=os_final_combined_data.drop(columns='label').columns)

In [24]:
X_os = os_final_combined_scaled_data
y_os = os_final_combined_data['label']

X_os_train, X_os_test, y_os_train, y_os_test = train_test_split(X_os, y_os, test_size=0.2, random_state=42, stratify=y_os)

powielenie klas malware (0), na danych treningowych

In [25]:
malware_train_data = X_os_train[y_os_train == 1]

num_samples_difference = len(X_os_train[y_os_train == 0]) - len(malware_train_data)
oversampled_malware_train_data = malware_train_data.sample(num_samples_difference, replace=True, random_state=42)

X_os_train_oversampled = pd.concat([X_os_train, oversampled_malware_train_data])
y_os_train_oversampled = pd.concat([y_os_train, pd.Series([1] * num_samples_difference)])

class_distribution_train = y_os_train_oversampled.value_counts()
print(class_distribution_train)

0    395308
1    395308
dtype: int64


In [26]:
os_n_features = os_final_combined_scaled_data.shape[1]

os_model = Sequential()
os_model.add(Dense(128, activation='relu', input_shape=(os_n_features,)))
os_model.add(Dropout(0.5))
os_model.add(Dense(64, activation='relu'))
os_model.add(Dropout(0.5))
os_model.add(Dense(1, activation='sigmoid'))

In [27]:
os_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Recall()])

In [28]:
os_history = os_model.fit(X_os_train_oversampled, y_os_train_oversampled, epochs=10, batch_size=32)

Epoch 1/10
24707/24707 [==============================] - 65s 3ms/step - loss: 0.3218 - recall: 0.8921
Epoch 2/10
24707/24707 [==============================] - 65s 3ms/step - loss: 0.2739 - recall: 0.9252
Epoch 3/10
24707/24707 [==============================] - 62s 3ms/step - loss: 0.2636 - recall: 0.9285
Epoch 4/10
24707/24707 [==============================] - 68s 3ms/step - loss: 0.2557 - recall: 0.9309
Epoch 5/10
24707/24707 [==============================] - 67s 3ms/step - loss: 0.2501 - recall: 0.9317
Epoch 6/10
24707/24707 [==============================] - 67s 3ms/step - loss: 0.2451 - recall: 0.9334
Epoch 7/10
24707/24707 [==============================] - 67s 3ms/step - loss: 0.2418 - recall: 0.9340
Epoch 8/10
24707/24707 [==============================] - 65s 3ms/step - loss: 0.2380 - recall: 0.9351
Epoch 9/10
24707/24707 [==============================] - 63s 3ms/step - loss: 0.2352 - recall: 0.9363
Epoch 10/10
24707/24707 [==============================] - 65s 3ms/step -

In [29]:
os_predictions = os_model.predict(X_os_test)

3120/3120 [==============================] - 4s 1ms/step


In [30]:
os_binary_predictions = (os_predictions > 0.5).astype(int)

## 5. Metryki ewaluacji modelu

In [31]:
accuracy = accuracy_score(y_os_test, os_binary_predictions)
print(f"accuracy {accuracy} ")

precision = precision_score(y_os_test, os_binary_predictions)
print(f"precision {precision} ")

recall = recall_score(y_os_test, os_binary_predictions)
print(f"recall {recall} ")

f1 = f1_score(y_os_test, os_binary_predictions)
print(f"f1 {f1} ")

conf_matrix = confusion_matrix(y_os_test, os_binary_predictions)
print(f"conf_matrix {conf_matrix} ")

accuracy 0.8579542608713073 
precision 0.062358879001195376 
recall 0.939 
f1 0.11695105243492339 
conf_matrix [[84708 14119]
 [   61   939]] 


### Teraz otrzymaliśmy dużo lepsze wyniki ponieważ pomimo małej starty w accuracy i ogromnej w precision, otrzymaliśmy już zadawalające wyniki na wykrywanie wirusów recall (94%) która w tym problemie jest dużo ważniejszą statystyką.

## 6. Możemy wyciągnąć wnioski na bazie następujących metryk:
*   Accuracy - około 86% wszystkich przypadków zostanie poprawnie sklasyfikowanych
*  Precision - Mówi nam o tym że tylko z wszystkich przypadków skalyfikowanym jako malware tylko 6% faktycznie była malware. Jest to trade off który został podjęty w związku z wysoką różnicą dystrybucji klas. W związku tym nasz model często będzie zwracał fałszywy alarm jednak w naszym przypadku jest to konieczne aby rzadko występujący wirus napewno się nie przedostał w naszeym problemie
*  Recall - Najważniejsza statystyka w naszym problemie. Model poprawnie identyfikuje 94% rzeczywistych przypadków malware. Jest to bardzo wysoki wynik, co jest kluczowe w kontekście wykrywania malware.
* F1-score - Mówi nam o słabej średniej harmonicznej między precission a recall



model prawdopodobnie będzie skutecznie wykrywał malware w danych produkcyjnych, ale może generować dużą liczbę fałszywych alarmów. Aby zminimalizować wpływ fałszywych alarmów,można rozważyć zmianę progu klasyfikacji. Jednak wszystko zależy od kontekstu zastosowania aplikacji, jeżeli ma dawać nam jedynie sygnały że coś jest niepokojącego warto jak najbarzdiej zwiększyć recall. Jeżeli ma jednak faktycznie blokować np. requesty warto jednak zwiększyć precision zmieniając np próg klasyfikacji na większy

## 3. Zbuduj pipeline przygotowujący dane do treningu / predykcji - jest to dla mnie nowy temat w prównaniu do powyżych rzeczy z którymi spotkałem się na studiach/projektach. Dlatego chciałbym go zrobić "obok" aby ewentualny błąd kardynalny tutaj nie wpłynął na ewentualną możliwość zatrudnienia ;)

In [32]:
benign_data = pd.read_csv(path_nb+'/CSV_benign.csv', low_memory=False)
malware_data = pd.read_csv(path_nb+'/CSV_malware.csv', low_memory=False)

benign_data['label'] = 0
malware_data['label'] = 1

combined_data = pd.concat([benign_data, malware_data], ignore_index=True)

categorical_features = combined_data.select_dtypes(include=['object']).columns.tolist()
numeric_features = combined_data.select_dtypes(exclude=['object']).columns.tolist()

In [33]:
categorical_transformer = Pipeline(
    [
        ('imputer_cat', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('ordinalencoder', OrdinalEncoder())
    ]
)

In [34]:
numeric_transformer = Pipeline(
    [
        ('imputer_num', SimpleImputer(strategy='most_frequent')),
        ('scaler', StandardScaler())
    ]
)

In [35]:
preprocessor = ColumnTransformer(
    [
        ('categoricals', categorical_transformer, categorical_features),
        ('numericals', numeric_transformer, numeric_features)
    ],
    remainder='drop'
)

In [36]:
pipeline = Pipeline(
    [
        ('preprocessing', preprocessor)
    ]
)

In [37]:
for col in categorical_features:
    combined_data[col] = combined_data[col].astype(str)

In [38]:
pipeline_data = pipeline.fit_transform(combined_data)

In [39]:
df_pipeline_data = pd.DataFrame(pipeline_data, columns=combined_data.columns)
df_pipeline_data.head(5)

,Country,ASN,TTL,IP,Domain,State,Registrant_Name,Country.1,Creation_Date_Time,hex_32,...,typos,oc_8,3gram,char_distribution,2gram,Registrar,sld,Name_Server_Count,Page_Rank,label
0,273.0,8730.0,68746.0,86645.0,776.0,4125.0,414.0,2653.0,0.0,0.0,...,0.0,76590.0,52423.0,76869.0,1790.0,64628.0,1088.0,-0.331992,-0.440166,-0.100582
1,284.0,7291.0,177854.0,86645.0,776.0,4125.0,414.0,2653.0,0.0,0.0,...,0.0,76590.0,52423.0,76869.0,1790.0,64628.0,1088.0,-0.370479,-0.440166,-0.100582
2,284.0,8130.0,177854.0,257909.0,5436.0,4125.0,1597.0,2653.0,0.0,0.0,...,0.0,76590.0,53193.0,76869.0,1789.0,64628.0,649.0,-0.370479,-0.440166,-0.100582
3,273.0,7291.0,68762.0,257909.0,5436.0,4125.0,1597.0,145658.0,0.0,0.0,...,0.0,76590.0,53193.0,76869.0,17052.0,64628.0,30626.0,-0.331992,-0.440166,-0.100582
4,284.0,12262.0,177854.0,72075.0,776.0,4125.0,414.0,2235.0,0.0,0.0,...,0.0,63695.0,32310.0,63945.0,2306.0,53691.0,649.0,-0.370479,-0.440166,-0.100582
